# Importing necessary Libraries and modules

In [2]:
!pip install contextualSpellCheck

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.6/282.6 kB 2.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.1/128.1 kB 5.7 MB/s eta 0:00:00


In [3]:
pip install pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 21.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install sentence-transformers annoy numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 1.5 MB/s eta 0:00:00a 0:00:01m
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install bert

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert: filename=bert-2.2.0-py3-none-any.whl size=3744 sha256=80d15835515b51d66b93f961313b7280dc6d0943af86b417a0bf66d41ef93088
  Stored in directory: /root/.cache/pip/wheels/d0/97/c8/5775d57c323970511488d9b53ce66cf0ded394fe4b4f6e0afb
  Created wheel for erlastic: filename=erlastic-2.0.0-py3-none-any.whl size=6779 sha256=f516378f5f19dc74feda95429e93641b1857769f153062d9dd7a76c78823a18a
  Stored in directory: /root/.cache/pip/wheels/f5/ca/b1/ce71c2e0e9e0df3786697d06d53574be03ef86cd4fb0e2cb99
Successfully built bert erlastic
Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import pandas as pd
import spacy
import re
import contextualSpellCheck
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from joblib import dump, load
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
from nltk.stem import PorterStemmer
from spacy.tokens import Doc
from spacy.language import Language
from spacy import displacy
from spacy.lang.en import English
from annoy import AnnoyIndex

# Task 1

In [7]:
# Function to Remove characters
def remove_characters(s):
  s_new=re.sub(r'[^A-Za-z0-9 ]+', '', s)
  return s_new

In [ ]:
# Load csv files
df_d= pd.read_csv("/kaggle/input/queries-data/docs.csv")
df_q=pd.read_csv("/kaggle/input/queries-data/queries.csv")
df_qd =pd.read_csv("/kaggle/input/queries-data/qdrel.csv")

In [ ]:
#  Remove all characters except alphnumeric and whitespaces from query text
df_q['query_text_M1']=df_q['query_text'].apply(remove_characters)

In [ ]:
# Remove all characters except alphnumeric and whitespaces from doc text
df_d['doc_text_M1']=df_d['doc_text'].apply(remove_characters)

In [8]:
# Loading spacy model
nlp = spacy.load('en_core_web_sm')
contextualSpellCheck.add_to_pipe(nlp)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [9]:
# Function for Spell checking
def spell_corrector(s):
  s_correct = nlp(s)
  if (len(s_correct._.suggestions_spellCheck) == 0):
    return s
  else :
    return s_correct._.outcome_spellCheck

In [ ]:
# spell corrector on a batch of data
def spell_corrector_batch(documents):
    return [spell_corrector(text) for text in documents]

# Function to tokenize a chunk of documents
def process_chunk(chunk):
    return spell_corrector_batch(tqdm(chunk))

In [ ]:
'''
parallel processing for spelling correction to each query by split the document list into chunck,
Apply tokenization to each document chunk and Combine the results from different chunks
'''

docs = df_d['doc_text_M1'].tolist()
num_processes = 4

chunk_size = len(docs) // num_processes
doc_chunks = [docs[i:i + chunk_size] for i in range(0, len(docs), chunk_size)]
with ProcessPoolExecutor(max_workers=num_processes) as executor:
    result_chunks = list(executor.map(process_chunk, doc_chunks))

df_d['doc_text_modify'] = [token for chunk in result_chunks for token in chunk]

In [ ]:
# parallel processing of for spelling correction to each document
# Assuming df_d['doc_text_modify'] contains a list of text documents
docs = df_q['query_text_M1'].tolist()

def spell_corrector_batch(documents):
    return [spell_corrector(text) for text in documents]

# Define the number of processes (adjust based on your system)
num_processes = 4

# Split the document list into chunks for parallel processing
chunk_size = len(docs) // num_processes
doc_chunks = [docs[i:i + chunk_size] for i in range(0, len(docs), chunk_size)]

# Function to tokenize a chunk of documents
def process_chunk(chunk):
    return spell_corrector_batch(chunk)

# Apply tokenization to each document chunk with parallel processing
with ProcessPoolExecutor(max_workers=num_processes) as executor:
    result_chunks = list(executor.map(process_chunk, doc_chunks))

# Combine the results from different chunks
df_q['query_text_modify'] = [token for chunk in result_chunks for token in chunk]

In [10]:
# Reload the save csv with spelling corrected words
df_d= pd.read_csv("/kaggle/input/query-modified-data/doc_modified.csv")
df_q=pd.read_csv("/kaggle/input/query-modified-data/query_modified.csv")
df_qd = pd.read_csv("/kaggle/input/queries-data/qdrel.csv")
df_d.sort_values(by=['doc_id'], inplace=True)
df_q.sort_values(by=['query_id'], inplace=True)
df_qd.sort_values(by=['query_id'], inplace=True)

In [11]:
for index, row in df_q.iterrows():
  print("Original Query: ",row['query_text'])
  print("Corrected Query: ", row['query_text_modify'])
  print("\n")

Original Query:  How does Quora look to a moderator?
Corrected Query:  How does Quora look to a man


Original Query:  How do I refuse to chose between different things to do in my life?
Corrected Query:  How do I refuse to chose between different things to do in my life


Original Query:  Did Ben Affleck shine more than Christian Bale as Batman?
Corrected Query:  Did Ben Affleck shine more than Christian Paul as?


Original Query:  What are the effects of demonitization of 500 and 1000 rupees notes on real estate sector?
Corrected Query:  What are the effects of inflation of 500 and 1000 paper notes on real estate sector


Original Query:  Why creativity is important?
Corrected Query:  Why creativity is important


Original Query:  How can I get free gems in Clash of Clans?
Corrected Query:  How can I get free time in Clash of Clans


Original Query:  How do you feel when someone upvotes your answer on Quora?
Corrected Query:  How do you feel when someone gives your answer on Quora




In [12]:
%%time
import spacy
from multiprocessing.pool import ThreadPool  
import multiprocessing as mp

nlp = spacy.load("en_core_web_sm")
docs = df_d['doc_text_modify'].tolist()
def tokenize(text):
    doc = nlp(str(text)) 
    return [token.text for token in doc]

# Params
num_workers = mp.cpu_count() * 2  
batch_size=100   
tokenized_documents=[]
with ThreadPool(num_workers) as executor:
    
    for i in range(0, len(docs), batch_size):

        batch = docs[i:i+batch_size]
        
        # Process in batches 
        tokenized_batch = executor.map(tokenize, batch)  
        
        # Save batch results
        tokenized_documents.extend(tokenized_batch) 
        
        # Clear batch to release memory
        del tokenized_batch

# print(tokenized_docs)

CPU times: user 1min 55s, sys: 34.1 s, total: 2min 29s
Wall time: 1min 41s


In [13]:
len(tokenized_documents)

10000

In [14]:
# # # parallel processing of to apply tokenization to each document
# # Assuming df_d['doc_text_modify'] contains a list of text documents
# from concurrent.futures import ProcessPoolExecutor
# from tqdm import tqdm

# docs = df_d['doc_text_modify'].tolist()

# def tokenize_document(text):
#     try:
#         doc = nlp(str(text))  # Ensure text is converted to string
#         return [token.text for token in doc]
#     except Exception as e:
#         print(f"Error tokenizing document: {e}")
#         return []

# # Define the number of processes (adjust based on your system)
# num_processes = 4

# # Split the document list into chunks for parallel processing
# chunk_size = len(docs) // num_processes
# doc_chunks = [docs[i:i + chunk_size] for i in range(0, len(docs), chunk_size)]

# # Function to tokenize a chunk of documents
# def process_chunk(chunk):
#     return [tokenize_document(text) for text in tqdm(chunk)]

# # Apply tokenization to each document chunk with parallel processing
# with ProcessPoolExecutor(max_workers=num_processes) as executor:
#     result_chunks = list(executor.map(process_chunk, doc_chunks))

# # Combine the results from different chunks
# tokenized_documents = [token for chunk in result_chunks for token in chunk]

In [17]:
# # parallel processing of to applt tokenization to each query
# # Assuming df_d['doc_text_modify'] contains a list of text documents
# queries = df_q['query_text_modify'].tolist()

# def tokenize_document(text):
#     try:
#         doc = nlp(str(text))  # Ensure text is converted to string
#         return [token.text for token in doc]
#     except Exception as e:
#         print(f"Error tokenizing document: {e}")
#         return []

# # Define the number of processes (adjust based on your system)
# num_processes = 4

# # Split the document list into chunks for parallel processing
# chunk_size = len(queries) // num_processes
# query_chunks = [queries[i:i + chunk_size] for i in range(0, len(queries), chunk_size)]

# # Function to tokenize a chunk of documents
# def process_chunk(chunk):
#     return [tokenize_document(text) for text in tqdm(chunk)]

# # Apply tokenization to each document chunk with parallel processing
# with ProcessPoolExecutor(max_workers=num_processes) as executor:
#     result_chunks = list(executor.map(process_chunk, query_chunks))

# # Combine the results from different chunks
# tokenized_queries = [token for chunk in result_chunks for token in chunk]

In [15]:
%%time
import spacy
from multiprocessing.pool import ThreadPool  
import multiprocessing as mp
nlp = spacy.load("en_core_web_sm")
query = df_q['query_text_modify'].tolist()
def tokenize(text):
    doc = nlp(str(text)) 
    return [token.text for token in doc]

# Params
num_workers = mp.cpu_count() * 2  
batch_size=100   
tokenized_queries=[]
with ThreadPool(num_workers) as executor:
    
    for i in range(0, len(query), batch_size):

        batch = query[i:i+batch_size]
        
        # Process in batches 
        tokenized_batch = executor.map(tokenize, batch)  
        
        # Save batch results
        tokenized_queries.extend(tokenized_batch) 
        
        # Clear batch to release memory
        del tokenized_batch

# print(tokenized_queries)

CPU times: user 2.41 s, sys: 353 ms, total: 2.76 s
Wall time: 2.29 s


In [16]:
all_tokens = [token for document_tokens in tokenized_documents for token in document_tokens]
# Remove rare and common words
from collections import Counter
token_counts = Counter(all_tokens)

min_doc_freq = 5
max_doc_freq_percent = 85

filtered_tokens = [token for token, count in token_counts.items() if min_doc_freq <= count <= (max_doc_freq_percent / 100) * len(tokenized_documents)]
# Create vocabulary
vocabulary = set(filtered_tokens)

# Join tokens back to form cleaned documents
cleaned_documents = [' '.join([token for token in doc_tokens if token in vocabulary]) for doc_tokens in tokenized_documents]

In [17]:
%%time
# Create TF-IDF vectorizer 
tfidf = TfidfVectorizer(vocabulary= vocabulary)
cleaned_tokens = [' '.join([token for token in query_tokens if token in vocabulary]) for query_tokens in tokenized_queries]
# Flatten the list of lists into a single list of strings
flattened_query_tokens = [(tokens) for tokens in cleaned_tokens]
# flattened_query_tokens = [" ".join(tokens) for tokens in tokenized_queries]
# Generate TF-IDF vectors

query_vectors = tfidf.fit_transform(flattened_query_tokens)

CPU times: user 6.77 ms, sys: 1.64 ms, total: 8.41 ms
Wall time: 18.5 ms


/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:1380: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [18]:
%%time
tfidf = TfidfVectorizer(vocabulary= vocabulary)
flattened_doc_tokens = [(tokens) for tokens in cleaned_documents]
doc_vectors = tfidf.fit_transform(flattened_doc_tokens)

CPU times: user 141 ms, sys: 0 ns, total: 141 ms
Wall time: 139 ms


In [19]:
# Function to get top n similar docs
def top_n_docs(doc_vector, query_vector, n):
    # Calculate cosine similarity
#     sim_matrix = cosine_similarity(query_vectors, doc_vectors)
    sim_matrix = cosine_similarity(query_vector, doc_vector)
#     indices = np.argsort(sim_matrix, axis=1)[:,-n:][:,::-1]
#     indices = np.argsort(sim_matrix)[-n:][::-1]
    indices = np.argsort(-sim_matrix)[:,:n]
    return indices

In [20]:
# Find top 5
top5 = top_n_docs(doc_vectors, query_vectors, 5) 

# Find top 10
top10 = top_n_docs(doc_vectors, query_vectors, 10)

# Print results
print("Top 5 most similar documents:")
for i, query_idx in enumerate(top5):
    print("Query", i + 1, "Index:", query_idx)
    print(df_d.iloc[query_idx])
    print()
    
print("Top 10 most similar documents:")
for i, query_idx in enumerate(top10):
    print("Query", i + 1, "Index:", query_idx)
    print(df_d.iloc[query_idx])
    print()
    

Top 5 most similar documents:
Query 1 Index: [9948 8614 4299 2178 3426]
      Unnamed: 0.1  Unnamed: 0  doc_id                               doc_text  \
9948          9948        9948   10268                How does a skunk spray?   
8614          8614        8614    8889    How does "Free" Translate to Latin?   
4299          4299        4299    4439                  How does a kiln work?   
2178          2178        2178    2253          How does one become a hacker?   
3426          3426        3426    3533  How does an aroused vagina feel like?   

                               doc_text_M1                    doc_text_modify  
9948                How does a skunk spray               How does a man spray  
8614      How does Free Translate to Latin    How does Free Translate to Love  
4299                  How does a kiln work                How does a man work  
2178          How does one become a hacker          How does one become a man  
3426  How does an aroused vagina feel lik

In [21]:
 def create_relevance_matrix(df_qd):
    # Extract unique query and document IDs
    unique_query_ids = df_qd['query_id'].unique()
    unique_doc_ids = df_qd['doc_id'].unique()

    # Create an empty relevance matrix filled with zeros
    relevance_matrix = np.zeros((len(unique_query_ids), 10000))

    # Iterate through each row in the CSV and update the relevance matrix
    for index, row in df_qd.iterrows():
        query_index = np.where(unique_query_ids == row['query_id'])[0][0]
        doc_index = np.where(unique_doc_ids == row['doc_id'])[0][0]
        relevance_matrix[query_index, doc_index] = 1

    # Convert the relevance matrix to a DataFrame for better visualization (optional)
#     relevance_df = pd.DataFrame(relevance_matrix, index=unique_query_ids, columns=unique_doc_ids)

    # Print or use the relevance matrix as needed
#     print(relevance_df)
    return relevance_matrix

In [22]:
'''
Function to extract row corresponding to query from relevance matrix and then find how many documents
amoung it are relevant as per given qdrel.csv
'''
def count_relevant_documents_for_query(query_index, top_k_indices, relevance_matrix):
    relevance_for_top_k = relevance_matrix[query_index, top_k_indices]
    count_ones = np.count_nonzero(relevance_for_top_k == 1)
    return count_ones

In [23]:
# Precision@k calculation
def precision_at_k(df_qd,  k, doc_vectors,query_vectors):
    precision_values = []
    relevance_matrix = create_relevance_matrix(df_qd)
    top_k_indices = top_n_docs(doc_vectors,query_vectors, k)
    for i in range(100):
        query_id= df_q['query_id'][i]
        top_k_indices_query=top_k_indices[i]
        
        # Calculate the count of relevant documents for the current query
        count_relevant = count_relevant_documents_for_query(i, top_k_indices_query, relevance_matrix)
        
        precision = count_relevant / min(k, len(top_k_indices[i]))
        precision_values.append(precision)
#         print(f"Query {query_id}: Number of relevant documents in top-k: {count_relevant}")
    avg_precision = np.mean(precision_values)
    print(f"Precision@{k}: {avg_precision:.6f}")
    return avg_precision

In [24]:
# Calculate Precision@1, Precision@5, Precision@10
precision_at_k(df_qd, 1, doc_vectors,query_vectors)
precision_at_k(df_qd, 5, doc_vectors,query_vectors)
precision_at_k(df_qd, 10, doc_vectors,query_vectors)

Precision@1: 0.000000
Precision@5: 0.000000
Precision@10: 0.001000


0.001

## Task 2 ##

## Stemming ##


spaCy doesn't contain any function for stemming as it relies on lemmatization only. Therefore, in this section, we will use NLTK for stemming.

In [25]:
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

# Load the English spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize the NLTK Porter Stemmer
stemmer = PorterStemmer()

# Define a custom pipeline component for stemming
@Language.component("custom_stemmer")
def custom_stemmer(doc):
    stemmed_tokens = [stemmer.stem(token.text) for token in doc]
    return Doc(doc.vocab, words=stemmed_tokens)

# Add the custom stemmer to the spaCy pipeline
nlp.add_pipe('custom_stemmer', name="stemmer", last=True)
queries = df_q['query_text_modify'].tolist()
num_processes = 4

# Split the document list into chunks for parallel processing
chunk_size = len(queries) // num_processes
query_chunks_s = [queries[i:i + chunk_size] for i in range(0, len(queries), chunk_size)]

# Function to tokenize a chunk of documents
def process_chunk(chunk):
    return [tokenize_document(text) for text in tqdm(chunk)]

# Function to tokenize a document using spaCy with stemming
def tokenize_document(text):
    try:
        doc = nlp(str(text))  # Ensure text is converted to string
        return [token.text for token in doc]
    except Exception as e:
        print(f"Error tokenizing document: {e}")
        return []

# Apply tokenization to each document chunk with parallel processing
with ProcessPoolExecutor(max_workers=num_processes) as executor:
    result_chunks_s = list(executor.map(process_chunk, query_chunks_s))

# Combine the results from different chunks
tokenized_queries_s = [token for chunk in result_chunks_s for token in chunk]

100%|██████████| 25/25 [00:00<00:00, 69.15it/s]


In [26]:
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

# Load the English spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize the NLTK Porter Stemmer
stemmer = PorterStemmer()

# Define a custom pipeline component for stemming
@Language.component("custom_stemmer")
def custom_stemmer(doc):
    stemmed_tokens = [stemmer.stem(token.text) for token in doc]
    return Doc(doc.vocab, words=stemmed_tokens)

# Add the custom stemmer to the spaCy pipeline
nlp.add_pipe('custom_stemmer', name="stemmer", last=True)

# Assuming df_d['doc_text_modify'] contains a list of text documents
docs = df_d['doc_text_modify'].tolist()

# Define the number of processes (adjust based on your system)
num_processes = 4

# Split the document list into chunks for parallel processing
chunk_size = len(docs) // num_processes
docs_chunks_s = [docs[i:i + chunk_size] for i in range(0, len(docs), chunk_size)]

# Function to tokenize a chunk of documents
def process_chunk(chunk):
    return [tokenize_document(text) for text in tqdm(chunk)]

# Function to tokenize a document using spaCy with stemming
def tokenize_document(text):
    try:
        doc = nlp(str(text))  # Ensure text is converted to string
        return [token.text for token in doc]
    except Exception as e:
        print(f"Error tokenizing document: {e}")
        return []

# Apply tokenization to each document chunk with parallel processing
with ProcessPoolExecutor(max_workers=num_processes) as executor:
    result_chunks_s = list(executor.map(process_chunk, docs_chunks_s))

# Combine the results from different chunks
tokenized_documents_s = [token for chunk in result_chunks_s for token in chunk]

100%|██████████| 2500/2500 [00:31<00:00, 79.46it/s] 


In [27]:
all_tokens_s = [token for document_tokens in tokenized_documents_s for token in document_tokens]
# Remove rare and common words

token_counts_s = Counter(all_tokens_s)

min_doc_freq = 5
max_doc_freq_percent = 85

filtered_tokens_s = [token for token, count in token_counts_s.items() if min_doc_freq <= count <= (max_doc_freq_percent / 100) * len(tokenized_documents_s)]
# Create vocabulary
vocabulary_stem = set(filtered_tokens_s)

# Join tokens back to form cleaned documents
cleaned_documents_s = [' '.join([token for token in doc_tokens if token in vocabulary_stem]) for doc_tokens in tokenized_documents_s]

In [28]:
%%time
# Create TF-IDF vectorizer 
tfidf = TfidfVectorizer(vocabulary= vocabulary_stem)
cleaned_tokens_s = [' '.join([token for token in query_tokens if token in vocabulary_stem]) for query_tokens in tokenized_queries_s]
# Flatten the list of lists into a single list of strings
flattened_query_tokens_s = [(tokens) for tokens in cleaned_tokens_s]
# flattened_query_tokens = [" ".join(tokens) for tokens in tokenized_queries]
# Generate TF-IDF vectors

query_vectors_s = tfidf.fit_transform(flattened_query_tokens_s)

CPU times: user 6.62 ms, sys: 1.89 ms, total: 8.51 ms
Wall time: 7.72 ms


/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:1380: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [29]:
%%time
tfidf = TfidfVectorizer(vocabulary= vocabulary_stem)
flattened_doc_tokens_s = [(tokens) for tokens in cleaned_documents_s]
doc_vectors_s = tfidf.fit_transform(flattened_doc_tokens_s)

CPU times: user 140 ms, sys: 1.99 ms, total: 142 ms
Wall time: 141 ms


In [30]:
print(doc_vectors_s.shape)
print(query_vectors_s.shape)

(10000, 2118)
(100, 2118)


In [31]:
# Calculate Precision@1, Precision@5, Precision@10
precision_at_k(df_qd, 1, doc_vectors_s,query_vectors_s)
precision_at_k(df_qd, 5, doc_vectors_s,query_vectors_s)
precision_at_k(df_qd, 10, doc_vectors_s,query_vectors_s)

Precision@1: 0.000000
Precision@5: 0.000000
Precision@10: 0.001000


0.001

## Lemmatizing ##

In [32]:
# parallel processing of to applt tokenization to each query
queries_l = df_q['query_text_modify'].tolist()
nlp = spacy.load("en_core_web_sm")
def tokenize_document_l(text):
#     try:
    doc = nlp(str(text))  # Ensure text is converted to string
    return [token.lemma_ for token in doc]
#     except Exception as e:
#         print(f"Error tokenizing document: {e}")
#         return []

# Define the number of processes (adjust based on your system)
num_processes = 4

# Split the document list into chunks for parallel processing
chunk_size_l = len(queries_l) // num_processes
query_chunks_l = [queries_l[i:i + chunk_size] for i in range(0, len(queries_l), chunk_size_l)]

# Function to tokenize a chunk of documents
def process_chunk_l(chunk):
    return [tokenize_document_l(text) for text in tqdm(chunk)]

# Apply tokenization to each document chunk with parallel processing
with ProcessPoolExecutor(max_workers=num_processes) as executor:
    result_chunks_l = list(executor.map(process_chunk_l, query_chunks_l))

# Combine the results from different chunks
tokenized_queries_l = [token for chunk in result_chunks_l for token in chunk]

100%|██████████| 100/100 [00:00<00:00, 107.21it/s]


In [34]:
def tokenize_document(text):
    try:
        doc = nlp(str(text))  # Ensure text is converted to string
        return [token.lemma_ for token in doc]
    except Exception as e:
        print(f"Error tokenizing document: {e}")
        return []
def process_chunk(chunk):
    return [tokenize_document(text) for text in tqdm(chunk)]

In [35]:
# # parallel processing of to apply tokenization to each document
docs_l = df_d['doc_text_modify'].tolist()
nlp = spacy.load("en_core_web_sm")
num_processes = 4

# Split the document list into chunks for parallel processing
chunk_size = len(docs) // num_processes
doc_chunks_l = [docs[i:i + chunk_size] for i in range(0, len(docs), chunk_size)]

# Apply tokenization to each document chunk with parallel processing
with ProcessPoolExecutor(max_workers=num_processes) as executor:
    result_chunks_l = list(executor.map(process_chunk, doc_chunks_l))

# Combine the results from different chunks
tokenized_documents_l = [token for chunk in result_chunks_l for token in chunk]

100%|██████████| 2500/2500 [00:29<00:00, 85.18it/s] 


In [36]:
all_tokens_l = [token for document_tokens in tokenized_documents_l for token in document_tokens]
# Remove rare and common words
token_counts_l = Counter(all_tokens_l)

min_doc_freq = 5
max_doc_freq_percent = 85

filtered_tokens_l = [token for token, count in token_counts_l.items() if min_doc_freq <= count <= (max_doc_freq_percent / 100) * len(tokenized_documents_l)]
# Create vocabulary
vocabulary_lemma = set(filtered_tokens_l)

# Join tokens back to form cleaned documents
cleaned_documents_l = [' '.join([token for token in doc_tokens if token in vocabulary_lemma]) for doc_tokens in tokenized_documents_l]

## Vocabulary sizes of Task1, Task2.1 and Task 2.2

In [37]:
print("Vocabulory size of Task 1: ",len(vocabulary))
print("Vocabulary size after Stemming: ",len(vocabulary_stem))
print("Vocabulary size after Lemmatizing: ",len(vocabulary_lemma))

Vocabulory size of Task 1:  2435
Vocabulary size after Stemming:  2118
Vocabulary size after Lemmatizing:  2091


In [38]:
%%time
# Create TF-IDF vectorizer 
tfidf = TfidfVectorizer(vocabulary= vocabulary_lemma)
cleaned_tokens_l = [' '.join([token for token in query_tokens if token in vocabulary_lemma]) for query_tokens in tokenized_queries_l]
# Flatten the list of lists into a single list of strings
flattened_query_tokens_l = [(tokens) for tokens in cleaned_tokens_l]
# flattened_query_tokens = [" ".join(tokens) for tokens in tokenized_queries]
# Generate TF-IDF vectors

query_vectors_l = tfidf.fit_transform(flattened_query_tokens_l)

CPU times: user 10.9 ms, sys: 1.17 ms, total: 12 ms
Wall time: 11.1 ms


In [39]:
%%time
tfidf = TfidfVectorizer(vocabulary= vocabulary_lemma)
flattened_doc_tokens_l = [(tokens) for tokens in cleaned_documents_l]
doc_vectors_l = tfidf.fit_transform(flattened_doc_tokens_l)

CPU times: user 141 ms, sys: 1.9 ms, total: 143 ms
Wall time: 142 ms


In [40]:
print(query_vectors_l[10])

  (0, 2077)	0.4493864358252359
  (0, 2044)	0.16612288203398426
  (0, 1905)	0.16809866330750334
  (0, 1904)	0.3113291330939607
  (0, 1414)	0.4493864358252359
  (0, 1375)	0.4493864358252359
  (0, 850)	0.4493864358252359
  (0, 805)	0.1471934967114523
  (0, 500)	0.13327264556891036


In [41]:
query_vectors_l.shape

(250, 2091)

In [ ]:
# Find top 5
top5 = top_n_docs(doc_vectors_l, query_vectors_l, 5) 

# Find top 10
top10 = top_n_docs(doc_vectors_l, query_vectors_l, 10)

# Print results
print("Top 5 most similar documents:")
for i, query_idx in enumerate(top5):
    print("Query", i + 1, "Index:", query_idx)
    print(df_d.iloc[query_idx])
    print()
    
print("Top 10 most similar documents:")
for i, query_idx in enumerate(top10):
    print("Query", i + 1, "Index:", query_idx)
    print(df_d.iloc[query_idx])
    print()

In [43]:
# Calculate Precision@1, Precision@5, Precision@10
precision_at_k(df_qd, 1, doc_vectors_l,query_vectors_l)
precision_at_k(df_qd, 5, doc_vectors_l,query_vectors_l)
precision_at_k(df_qd, 10, doc_vectors_l,query_vectors_l)

Precision@1: 0.000000
Precision@5: 0.000000
Precision@10: 0.001000


0.001

## Task 3 ##

## NER and POS ##

In [44]:
def get_nouns_and_entities(text):
    try:
        doc = nlp(str(text))
        nouns = [token.lemma_ for token in doc if token.pos_ == "NOUN"]
        entities = [ent.text for ent in doc.ents]
        return nouns, entities
    except Exception as e:
        print(f"Error processing document: {e}")
        return [], []
def process_chunk(chunk):
    return [get_nouns_and_entities(text) for text in tqdm(chunk)]

In [45]:
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
queries_p = flattened_query_tokens_l
num_processes = 4

# Split the document list into chunks for parallel processing
chunk_size = len(queries_p) // num_processes
query_chunks_p = [queries_p[i:i + chunk_size] for i in range(0, len(queries_p), chunk_size)]

# Apply processing to each document chunk with parallel processing
with ProcessPoolExecutor(max_workers=num_processes) as executor:
    result_chunks_p = list(executor.map(process_chunk, query_chunks_p))

# Combine the results from different chunks
q_nouns, q_entities = zip(*[item for chunk in result_chunks_p for item in chunk])

100%|██████████| 2/2 [00:00<00:00, 76.38it/s]


In [46]:
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

# Assuming df_q['query_text_modify'] contains a list of text queries
docs_p = flattened_doc_tokens_l

# Define the number of processes (adjust based on your system)
num_processes = 4

# Split the document list into chunks for parallel processing
chunk_size = len(docs_p) // num_processes
doc_chunks_p = [docs_p[i:i + chunk_size] for i in range(0, len(docs_p), chunk_size)]

# Apply processing to each document chunk with parallel processing
with ProcessPoolExecutor(max_workers=num_processes) as executor:
    result_chunks_p = list(executor.map(process_chunk, doc_chunks_p))

# Combine the results from different chunks
d_nouns, d_entities = zip(*[item for chunk in result_chunks_p for item in chunk])

100%|██████████| 2500/2500 [00:28<00:00, 86.61it/s] 


In [47]:
# Function to modify TF-IDF vectors based on entities
def modify_tfidf_with_entities(tfidf_vectors, entities_list, vectorizer, multiplier):
    modified_vectors = tfidf_vectors.copy()
    feature_names = np.array(vectorizer.get_feature_names_out())

    for i, entities in enumerate(entities_list):
        for entity in entities:
            # Find the index of the entity in the vocabulary
            vocab_index = np.where(feature_names == entity)[0]
            if len(vocab_index) > 0:
                # Multiply the TF-IDF value by the specified multiplier
                modified_vectors[i, vocab_index] *= multiplier

    return modified_vectors

tfidf_vectorizer = TfidfVectorizer(vocabulary= vocabulary_lemma)
# Modify query TF-IDF vectors
modified_query_n = modify_tfidf_with_entities(query_vectors_l, q_nouns,tfidf_vectorizer, 2)

# Modify document TF-IDF vectors
modified_doc_n = modify_tfidf_with_entities(doc_vectors_l, d_nouns,tfidf_vectorizer, 2)

# You can use modified_query_tfidf and modified_doc_tfidf for further analysis

In [48]:
print(modified_query_n.shape)
print(modified_doc_n.shape)

(250, 2091)
(10000, 2091)


In [49]:
query_vectors_weighted = modify_tfidf_with_entities(modified_query_n, q_entities,tfidf_vectorizer, 4)

# Modify document TF-IDF vectors
doc_vectors_weighted = modify_tfidf_with_entities(modified_doc_n, d_entities,tfidf_vectorizer, 4)

In [50]:
print(query_vectors_l[36])

  (0, 2086)	0.38418233517341244
  (0, 2044)	0.2243351352239836
  (0, 2036)	0.5347361384565494
  (0, 1991)	0.3972057230927248
  (0, 893)	0.5646698325021522
  (0, 805)	0.19877257476245505


In [51]:
print(modified_query_n[36])

  (0, 2086)	0.38418233517341244
  (0, 2044)	0.2243351352239836
  (0, 2036)	1.0694722769130989
  (0, 1991)	0.3972057230927248
  (0, 893)	1.1293396650043044
  (0, 805)	0.19877257476245505


In [52]:
print(query_vectors_weighted[36])

  (0, 2086)	0.38418233517341244
  (0, 2044)	0.2243351352239836
  (0, 2036)	1.0694722769130989
  (0, 1991)	0.3972057230927248
  (0, 893)	1.1293396650043044
  (0, 805)	0.19877257476245505


In [53]:
# Calculate Precision@1, Precision@5, Precision@10
precision_at_k(df_qd, 1, doc_vectors_weighted,query_vectors_weighted)
precision_at_k(df_qd, 5, doc_vectors_weighted,query_vectors_weighted)
precision_at_k(df_qd, 10, doc_vectors_weighted,query_vectors_weighted)

Precision@1: 0.000000
Precision@5: 0.000000
Precision@10: 0.001000


0.001

## Task 4 ##

In [55]:
import bert
from spellchecker import SpellChecker
from nltk.tokenize import word_tokenize

In [56]:
spell_checker = SpellChecker()
def preprocess_text_spell_correction(text):
    # Perform text preprocessing steps
    # For example, remove special characters, lowercase, and perform spell-checking
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase

    # Perform spell-checking
    words = text.split()
    corrected_words = [spell_checker.correction(word) for word in words if spell_checker.correction(word) is not None]
    text = ' '.join(corrected_words)

    return text

In [57]:
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

# Assuming df_q['query_text'] contains a list of text queries
queries = df_q['query_text'].tolist()

def preprocess_text_spell_correction(text):
    try:
        text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
        text = text.lower()  # Convert to lowercase

        # Perform spell-checking
        words = text.split()
        corrected_words = [spell_checker.correction(word) for word in words if spell_checker.correction(word) is not None]
        text = ' '.join(corrected_words)

        return text
    except Exception as e:
        print(f"Error preprocessing text: {e}")
        return ''

# Define the number of processes (adjust based on your system)
num_processes = 4

# Split the query list into chunks for parallel processing
chunk_size = len(queries) // num_processes
query_chunks_i = [queries[i:i + chunk_size] for i in range(0, len(queries), chunk_size)]

# Function to preprocess a chunk of queries
def preprocess_chunk(chunk):
    return [preprocess_text_spell_correction(query) for query in tqdm(chunk)]

# Apply preprocessing to each query chunk with parallel processing
with ProcessPoolExecutor(max_workers=num_processes) as executor:
    preprocessed_query_chunks_i = list(executor.map(preprocess_chunk, query_chunks_i))

# Combine the results from different chunks
queries_i = [item for chunk in preprocessed_query_chunks_i for item in chunk]


100%|██████████| 25/25 [00:16<00:00,  1.56it/s]


In [58]:
from sentence_transformers import SentenceTransformer, util
# Get BERT embeddings for queries and documents
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [59]:
query_embeddings = model.encode(queries_i, convert_to_tensor=True)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

# Assuming df_q['query_text'] contains a list of text queries
docs = df_d['doc_text'].tolist()

def preprocess_text_spell_correction(text):
    try:
        text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
        text = text.lower()  # Convert to lowercase

        # Perform spell-checking
        words = text.split()
        corrected_words = [spell_checker.correction(word) for word in words if spell_checker.correction(word) is not None]
        text = ' '.join(corrected_words)

        return text
    except Exception as e:
        print(f"Error preprocessing text: {e}")
        return ''

# Define the number of processes (adjust based on your system)
num_processes = 4

# Split the query list into chunks for parallel processing
chunk_size = len(docs) // num_processes
docs_chunks_i = [docs[i:i + chunk_size] for i in range(0, len(docs), chunk_size)]

# Function to preprocess a chunk of queries
def preprocess_chunk(chunk):
    return [preprocess_text_spell_correction(query) for query in tqdm(chunk)]

# Apply preprocessing to each query chunk with parallel processing
with ProcessPoolExecutor(max_workers=num_processes) as executor:
    preprocessed_docs_chunks_i = list(executor.map(preprocess_chunk, docs_chunks_i))

# Combine the results from different chunks
docs_i = [item for chunk in preprocessed_docs_chunks_i for item in chunk]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  0%|          | 0/2500 [00:

In [ ]:
docs_embeddings = model.encode(docs_i, convert_to_tensor=True)

In [ ]:
%%time
num_trees = 100  
dense_doc_vectors = docs_embeddings
annoy_index = AnnoyIndex(dense_doc_vectors.shape[1], metric='angular')  

for i, v in enumerate(dense_doc_vectors):
    annoy_index.add_item(i, v)

annoy_index.build(num_trees) 

In [ ]:
query_embeddings.shape

In [ ]:
k = 5
top_k_indices = []
for query_embedding in query_embeddings:
    indices, _ = annoy_index.get_nns_by_vector(query_embedding.cpu().numpy(), k, include_distances=True)
    top_k_indices.append(indices)
top_k_indices_matrix = np.array(top_k_indices)

In [ ]:
# Precision@k calculation
def precision_at_k_modify(df_qd,  k, doc_vectors,query_vectors, top_k_indices):
    precision_values = []
    relevance_matrix = create_relevance_matrix(df_qd)
    for i in range(100):
        # Extract relevant information for the current query
        query_id= df_q['query_id'][i]
        top_k_indices_query=top_k_indices[i]
        # Calculate the count of relevant documents for the current query
        count_relevant = count_relevant_documents_for_query(i, top_k_indices_query, relevance_matrix)
        precision = count_relevant / min(k, len(top_k_indices[i]))
        precision_values.append(precision)
#         print(f"Query {query_id}: Number of relevant documents in top-k: {count_relevant}")
    avg_precision = np.mean(precision_values)
    print(f"Precision@{k}: {avg_precision:.6f}")
    return avg_precision

In [ ]:
# Calculate Precision@1, Precision@5, Precision@10
precision_at_k_modify(df_qd, 1, docs_embeddings,query_embeddings, top_k_indices_matrix)
precision_at_k_modify(df_qd, 5, docs_embeddings,query_embeddings, top_k_indices_matrix)
precision_at_k_modify(df_qd, 10, docs_embeddings,query_embeddings, top_k_indices_matrix)